In [1]:
import nltk
import torch
import tensorflow as tf
import keras
import warnings
warnings.simplefilter('ignore')

/home/cloud/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/cloud/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/cloud/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/cloud/anaconda/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:5

In [1]:
!pip3 install -U textblob
!pip3 install -U rake-nltk
!python3 -m textblob.download_corpora

Requirement already up-to-date: textblob in /home/cloud/anaconda/lib/python3.6/site-packages (0.15.3)
Requirement already up-to-date: rake-nltk in /home/cloud/anaconda/lib/python3.6/site-packages (1.0.4)
[nltk_data] Downloading package brown to /home/cloud/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/cloud/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/cloud/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/cloud/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package conll2000 to /home/cloud/nltk_data...
[nltk_data]   Package conll2000 is already up-to-date!
[nltk_data] Downloading package movie_reviews to
[nltk_data]     /home/cloud/nltk_data...
[nltk_data]   Package movie_revi

In [2]:
text = []

In [3]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [14]:
def preprocess(txt): #convert to lower case, remove stopwords, get important entities
    
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize as tokenize
    
    stop_words=set(stopwords.words('english'))
    t_list, key_phrases, entities = [],[],[]
    txt_tokens=tokenize(txt)
    
    t_list = [word.lower() for word in txt_tokens if not word in stop_words]
#     print("After removing stopwords :\n",t_list) #return this
    
    pre_processed_string = ""
    for word in t_list :
        pre_processed_string=pre_processed_string+word+" "
    pre_processed_string.strip()
#     print("Preprocessed String : \n",pre_processed_string)
    
    from rake_nltk import Rake
    
    r=Rake()
    r.extract_keywords_from_text(txt)
    key_phrases = r.get_ranked_phrases()
    for item in key_phrases:
#         print(item)
        if item.split(" ")[0]==str(item):
            key_phrases.remove(item)
    if len(key_phrases) > 20:
        for i in range(len((key_phrases))//3):
            key_phrases.pop(0)
#     print("Key Phrases : \n",key_phrases) #return this
    
    import spacy
    from spacy import displacy
    from collections import Counter
    import en_core_web_sm
    nlp = en_core_web_sm.load()
    
    doc = nlp(txt)
    entities = [(X.text, X.label_) for X in doc.ents]
#     if len(entities) != 0:
#         print("Entities : \n",entities) #return this
#     else:
#         print("No Entities")
    
    return key_phrases

In [5]:
def information_from_article(url):
    
    import spacy
    import requests
    import re
    from bs4 import BeautifulSoup
    
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))


In [6]:
def get_information(url):
    
    import warnings
    warnings.simplefilter('ignore')
    extractor=information_from_article(url)
    article = nlp(extractor)
    print("Number of entities in the page : ",len(article.ents)) #return this
    labels = [x.label_ for x in article.ents]
    
    i = len(labels)//3 #30% of the top label count
    items = [x.text for x in article.ents]
    common_items = [] #Counter(items).most_common(i)
    too_common=[x for x in range(5)]
    for item in Counter(items).most_common(i):
        try:
            if int(str(item).split(',')[1].split(')')[0]) in too_common:
                continue
            else:
                common_items.append(str(item).split(',')[0])
        except ValueError as e:
            pass
        
    sentences = [x for x in article.sents]
    print(len(sentences))
    print("Most important terms in the document : \n",common_items) #return this
    
    for i in range(len(sentences)):
        displacy.render(nlp(str(sentences[i])), jupyter=True, style='ent')


In [7]:
get_information("https://medium.com/@jonathan_hui/gan-why-it-is-so-hard-to-train-generative-advisory-networks-819a86b3750b")

Number of entities in the page :  119
178
Most important terms in the document : 
 ["('GAN'", "('KL'", "('0'", "('zero'", "('Arjovsky'"]


In [8]:
preprocess("This is probably the most popular supermarket in Tippasandra. One can get most of the grocery items,  household cleaning items,  cookware, kitchen appliances and much more.Prices are competitive and the staff is helpful.There is no parking available at this place and it is located on a busy street so you might have to look for parking on adjoining streets.")

Entities : 
 [('Tippasandra', 'GPE')]


['household cleaning items',
 'grocery items',
 'popular supermarket',
 'kitchen appliances',
 'busy street',
 'adjoining streets',
 'parking available',
 'tippasandra',
 'probably',
 'place',
 'much',
 'look',
 'helpful',
 'cookware']

In [9]:
!pip3 install google
!pip3 install beautifulsoup4

In [10]:
def scrape_reviews(query):
    try:
        from googlesearch import search
    except ImportError:
        print("No module named 'google'")
    formulated_query=""
    for item in query.split(" "):
        if item==query.split(" ")[0]:
            formulated_query=item
        else:
            formulated_query=formulated_query+"+"+item
    formulated_query = formulated_query+"+google+reviews"
#     print(formulated_query)
#     tld="com/search?q="+query.split(" ")[0]+"+"+query.split(" ")[1]+"+google+reviews"
    for j in search(formulated_query, tld="com", num=10, pause=1, stop=1):
        print("Link : ",str(j))
        new_data=information_from_article(str(j))
        new_data.strip();
        preprocess(new_data)

In [11]:
scrape_reviews("leon grill")

Link :  https://www.yelp.com/biz/leon-grill-burritos-and-margaritas-maineville-2
Entities : 
 [('Yelp Reviews', 'PERSON'), ('Leon Grill Burritos', 'ORG'), ('35', 'CARDINAL'), ('Photos - (New', 'ORG'), ('22', 'CARDINAL'), ('3', 'CARDINAL'), ('Maineville', 'LOC'), ('Search Form                                                                                                     ', 'ORG'), ('Navigation                                                                                                     ', 'ORG'), ('Page Content', 'ORG'), ('First', 'ORDINAL'), ('Current Location', 'ORG'), ('Allow', 'PERSON'), ('Grant Permissions', 'PERSON'), ('Chrome', 'WORK_OF_ART'), ('Secure', 'PRODUCT'), ('Location', 'ORG'), ('Ask', 'PERSON'), ('Allow', 'PERSON'), ('Yelp', 'PERSON'), ('Google', 'ORG'), ('Opera', 'PRODUCT'), ('Yelp', 'PERSON'), ('Opera', 'ORG'), ('Click Safari', 'PERSON'), ('the Menu Bar', 'FAC'), ('Click', 'PERSON'), ('Prompt', 'PERSON'), ('Prompt', 'PERSON'), ('one', 'CARDINAL'), ('MacOS',

In [25]:
def form_phrases_for_wordnet(filename):
    file=open(filename,"r")
    review_phrases= []
    store=""
    for sent in file.readlines():
        if "store = " in sent:
            print("Store : ",sent[8:])
            store=sent[8:]
#             print("Reviews : ")
        else:
            review_phrases.append(preprocess(sent))
    return review_phrases,store

In [27]:
wordnet,store=[],""
wordnet, store = form_phrases_for_wordnet("input0.txt")
for item in wordnet:
    print(wordnet)

Store :  noble super bazar

[[], ['ok ok supermarket', 'friendly'], ['good spices collection including', 'west india'], ['expired food items', 'old stock', 'good variety', 'lot', 'buying'], ['even home delivery', 'branded food items', 'food coupons', 'really helpful', 'product range', 'one stop', 'credit card', 'also accepted', 'people', 'huge', 'available'], ['help big time', 'super store', 'grocery purchase', 'dark inside', 'renovated'], ['staff seem', 'online booking', 'good experience', 'dunzo app', 'store', 'introduction', 'better'], ['online shopping', 'good place', 'direct'], ['household cleaning items', 'grocery items', 'popular supermarket', 'kitchen appliances', 'probably', 'much', 'cookware'], ['prices', 'competitive'], ['busy street', 'adjoining streets', 'parking available', 'place', 'look'], ['main road easily accessible', 'located', 'almost'], ['good place', 'day shopping'], ['good supermarket', 'decent selection'], ['variou items display signs', 'local provision stores'